Guidance on using this dataset: https://www.kaggle.com/code/acelevin/identifying-playing-cards

Download dataset from: https://www.kaggle.com/datasets/gunhcolab/object-detection-dataset-standard-52card-deck/data

In [31]:
import tensorflow as tf
import pickle
from PIL import Image
import os
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt

In [46]:
#hyper parameters:
BATCH_SIZE = 32
NUM_EPOCHS = 10

In [7]:
# Load in data
train_labels = pd.read_csv("data/train_labels.csv")
test_labels = pd.read_csv("data/test_labels.csv")

# change class labels from str to int
name_to_int_dict = {'ace of spades': 0, 'two of spades':1, 'three of spades':2, 'four of spades':3, 
                    'five of spades':4, 'six of spades':5, 'seven of spades':6, 'eight of spades':7, 
                    'nine of spades':8, 'ten of spades':9, 'jack of spades':10, 'queen of spades':11, 
                    'king of spades':12, 'ace of hearts': 13, 'two of hearts': 14, 'three of hearts': 15, 
                    'four of hearts': 16, 'five of hearts': 17, 'six of hearts': 18, 'seven of hearts':19, 
                    'eight of hearts':20, 'nine of hearts':21, 'ten of hearts':22, 'jack of hearts': 23, 
                    'queen of hearts': 24, 'king of hearts':25, 'ace of clubs': 26, 'two of clubs': 27, 
                    'three of clubs': 28, 'four of clubs': 29, 'five of clubs': 30, 'six of clubs': 31,
                    'seven of clubs':32, 'eight of clubs':33, 'nine of clubs':34, 'ten of clubs':35, 
                    'jack of clubs': 36, 'queen of clubs': 37, 'king of clubs':38, 'ace of diamonds': 39, 
                    'two of diamonds': 40, 'three of diamonds': 41, 'four of diamonds': 42, 'five of diamonds': 43, 
                    'six of diamonds': 44,'seven of diamonds':45, 'eight of diamonds':46, 'nine of diamonds':47, 
                    'ten of diamonds':48, 'jack of diamonds': 49, 'queen of diamonds': 50,'king of diamonds':51,
                    
                    # these are to account for some typos in the dataset
                    'three of dimaonds': 41, 'four of dimaonds': 42, 'five of dimaonds': 43, 'six of dimaonds': 44, 
                    'eigth of clubs':33, 'seven of seven':32 
                    }

test_labels['class_number'] = test_labels['class'].map(name_to_int_dict)
train_labels['class_number'] = train_labels['class'].map(name_to_int_dict)

# These should print empty to show that all labels are accounted for
print(test_labels.loc[test_labels['class_number'].isnull()])
print(train_labels.loc[train_labels['class_number'].isnull()])

image_paths_train = train_labels['filename'].values
labels_train = train_labels['class_number'].values

image_paths_test = test_labels['filename'].values
labels_test = test_labels['class_number'].values

Empty DataFrame
Columns: [filename, width, height, class, xmin, ymin, xmax, ymax, class_number]
Index: []
Empty DataFrame
Columns: [filename, width, height, class, xmin, ymin, xmax, ymax, class_number]
Index: []


In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices((image_paths_train, labels_train))
test_dataset = tf.data.Dataset.from_tensor_slices((image_paths_test, labels_test))

def load_train_image(image_path, label):
    image_path = tf.strings.join(['train/', image_path], separator='')
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [300, 200])
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

def load_test_image(image_path, label):
    image_path = tf.strings.join(['test/', image_path], separator='')
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [300, 200])
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_dataset = train_dataset.map(load_train_image, num_parallel_calls=tf.data.AUTOTUNE).shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.map(load_test_image, num_parallel_calls=tf.data.AUTOTUNE).shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [40]:
class CardPredictor(tf.keras.Model):
    def __init__(self):
        super(CardPredictor, self).__init__()
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
        
        data_augmentation = tf.keras.models.Sequential([
                                tf.keras.layers.RandomRotation(0.1),
                                tf.keras.layers.RandomZoom(0.1),
                            ])
        
        self.architecture = [        
                tf.keras.layers.InputLayer((300, 300, 3)),
                data_augmentation,
                             
                tf.keras.layers.Conv2D(32, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),

                tf.keras.layers.Dropout(0.3),
                tf.keras.layers.Conv2D(64, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),

                tf.keras.layers.Dropout(0.3),
                tf.keras.layers.Conv2D(128, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),
                
                tf.keras.layers.Dropout(0.3),
                tf.keras.layers.Conv2D(256, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),
                
                tf.keras.layers.Dropout(0.3),
                tf.keras.layers.Conv2D(512, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),

                tf.keras.layers.Flatten(),                                   # Flatten to vector
                
                tf.keras.layers.Dense(256, activation='relu'),               # Fully connected layer
                tf.keras.layers.Dropout(0.5),                                # Prevent overfitting
                tf.keras.layers.Dense(52, activation='softmax')
                ]
        
        self.sequential = tf.keras.Sequential(self.architecture, name="card_predictor")
        
    def call(self, x):
        """ Passes input image through the network. """
        return self.sequential(x)

    @staticmethod
    def loss_fn(labels, predictions): 
           """ Loss function for the model. """
           return tf.keras.losses.sparse_categorical_crossentropy(labels, predictions)

In [33]:
with open('train.pkl', 'rb') as file:
    data = pickle.load(file)
    
new_data = {}
for key, inner_dict in data.items():
    img_path = inner_dict['img_path']
    value = inner_dict['class_label']
    new_data[img_path] = value

In [44]:
dataset = tf.data.Dataset.from_tensor_slices((new_data.keys(), new_data.values()))

def load_train_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [300, 300])
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

dataset = dataset.map(load_train_image, num_parallel_calls=tf.data.AUTOTUNE).shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [47]:
model = CardPredictor()
model.compile(optimizer=model.optimizer, loss=model.loss_fn, metrics=['accuracy'])
model.fit(dataset, epochs=NUM_EPOCHS, verbose=1)

Epoch 1/10
1397/1397 ━━━━━━━━━━━━━━━━━━━━ 82s 56ms/step - accuracy: 0.1729 - loss: 2.7336
Epoch 2/10
1397/1397 ━━━━━━━━━━━━━━━━━━━━ 83s 58ms/step - accuracy: 0.3577 - loss: 1.9565
Epoch 3/10
1397/1397 ━━━━━━━━━━━━━━━━━━━━ 80s 57ms/step - accuracy: 0.5186 - loss: 1.4005
Epoch 4/10
1397/1397 ━━━━━━━━━━━━━━━━━━━━ 82s 57ms/step - accuracy: 0.6581 - loss: 1.0200
Epoch 5/10
1397/1397 ━━━━━━━━━━━━━━━━━━━━ 82s 58ms/step - accuracy: 0.7517 - loss: 0.7927
Epoch 6/10
1397/1397 ━━━━━━━━━━━━━━━━━━━━ 81s 57ms/step - accuracy: 0.8184 - loss: 0.6132
Epoch 7/10
1397/1397 ━━━━━━━━━━━━━━━━━━━━ 83s 58ms/step - accuracy: 0.8495 - loss: 0.5865
Epoch 8/10
1397/1397 ━━━━━━━━━━━━━━━━━━━━ 82s 57ms/step - accuracy: 0.8822 - loss: 0.4440
Epoch 9/10
1397/1397 ━━━━━━━━━━━━━━━━━━━━ 81s 57ms/step - accuracy: 0.8834 - loss: 0.4876
Epoch 10/10
1397/1397 ━━━━━━━━━━━━━━━━━━━━ 81s 57ms/step - accuracy: 0.8910 - loss: 0.5651


In [49]:
model.save('model_weights_89.keras')  # can also use .h5 extension if preferred